In [20]:
import os
import numpy as np
import pandas as pd
import joblib
import torch
import time
from sklearn.preprocessing import StandardScaler

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("="*80)
print(f"🚀 Starting Test Inference | Device: {device.upper()}")
start_time = time.time()
print("="*80)

🚀 Starting Test Inference | Device: CUDA


In [22]:
test_csv_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\test_processed.csv"
embedding_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\test_cnn_features_efficientnetb3.npy"

cat_model_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\CatBoost_model_gpu.pkl"
lgb_model_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\LightGBM_model_gpu.pkl"
xgb_model_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\XGBoost_model_gpu.pkl"


In [23]:
weights = {"CatBoost": 0.2, "LightGBM": 0.4, "XGBoost": 0.4}

In [24]:
print("📥 Loading test data and embeddings...")
test_df = pd.read_csv(test_csv_path)
features = np.load(embedding_path, allow_pickle=True)
print(f"✅ Loaded embeddings shape: {features.shape}")


📥 Loading test data and embeddings...
✅ Loaded embeddings shape: (75000, 1536)


In [25]:
exclude_cols = [
    "sample_id", "image_link", "catalog_content",
    "item_name", "bullet_points", "product_description",
    "text_all"  # remove long text columns
]


In [26]:
exclude_cols = ["sample_id", "image_link"]
X_tab = test_df.drop(columns=exclude_cols).values
print(f"✅ Tabular features shape: {X_tab.shape}")

✅ Tabular features shape: (75000, 9)


In [28]:
X_tab = test_df.drop(columns=[c for c in exclude_cols if c in test_df.columns])
X_tab = X_tab.select_dtypes(include=[np.number])  # ensure numeric only

print(f"✅ Tabular numeric features shape: {X_tab.shape}")

✅ Tabular numeric features shape: (75000, 2)


In [29]:
scaler = StandardScaler()
X_tab_scaled = scaler.fit_transform(X_tab)

# Merge tabular + CNN embeddings
X_test_final = np.hstack([X_tab_scaled, features])
print(f"✅ Final merged feature shape: {X_test_final.shape}")

✅ Final merged feature shape: (75000, 1538)


In [30]:
print("\n⚙️ Loading trained models...")
cat_model = joblib.load(cat_model_path)
lgb_model = joblib.load(lgb_model_path)
xgb_model = joblib.load(xgb_model_path)
print("✅ All models loaded successfully!")


⚙️ Loading trained models...
✅ All models loaded successfully!


In [32]:
# %% --- Final merge and shape check ---
X_test_final = np.hstack([X_tab_scaled, features])

print("✅ Final test shape:", X_test_final.shape)

# Ensure same number of features as training
expected_features = cat_model.feature_importances_.shape[0]
if X_test_final.shape[1] != expected_features:
    print(f"⚠️ Shape mismatch! Expected {expected_features}, got {X_test_final.shape[1]}")
else:
    print("✅ Shapes match perfectly with training!")

# %% --- Safe prediction ---
print("\n🔮 Generating predictions... (this may take a few minutes)")
try:
    cat_preds = cat_model.predict(X_test_final)
    lgb_preds = lgb_model.predict(X_test_final)
    xgb_preds = xgb_model.predict(X_test_final)

    final_preds = (
        weights["CatBoost"] * cat_preds +
        weights["LightGBM"] * lgb_preds +
        weights["XGBoost"] * xgb_preds
    )

    # Reverse log transform (since you trained on log(price))
    final_preds = np.expm1(final_preds)

    # Save submission
    submission = pd.DataFrame({
        "sample_id": test_df["sample_id"],
        "predicted_price": final_preds
    })
    submission.to_csv("final_predictions_with_image.csv", index=False)
    print("✅ Final predictions saved to final_predictions_with_image.csv")

except Exception as e:
    print(f"❌ Prediction failed: {e}")


✅ Final test shape: (75000, 1538)
⚠️ Shape mismatch! Expected 1630, got 1538

🔮 Generating predictions... (this may take a few minutes)
❌ Prediction failed: catboost/libs/data/model_dataset_compatibility.cpp:72: Feature 1538 is present in model but not in pool.


In [33]:
print("Model expected features:", cat_model.feature_importances_.shape[0])
print("Test features shape:", X_test_final.shape[1])


Model expected features: 1630
Test features shape: 1538


In [41]:
import numpy as np
import pandas as pd

# ✅ Load training data
train_embeddings = np.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\cnn_features_efficientnetb3_torch.npy")
train_tabular = pd.read_csv(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\train_processed.csv")

# Exclude non-feature columns
exclude_cols = ["sample_id", "image_link", "price"]
train_tabular = train_tabular.drop(columns=exclude_cols)

# ✅ Load test data
test_embeddings = np.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\test_cnn_features_efficientnetb3.npy")
test_tabular = pd.read_csv(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\test_processed.csv")
test_tabular = test_tabular.drop(columns=["sample_id", "image_link"])

# ✅ Combine embeddings + tabular
X_train_final = np.hstack([train_embeddings, train_tabular.values])
X_test_final = np.hstack([test_embeddings, test_tabular.values])

print("Train embeddings shape:", train_embeddings.shape)
print("Train tabular shape:", train_tabular.shape)
print("Test embeddings shape:", test_embeddings.shape)
print("Test tabular shape:", test_tabular.shape)
print("\n✅ Train final shape:", X_train_final.shape)
print("✅ Test final shape:", X_test_final.shape)


Train embeddings shape: (75000, 1536)
Train tabular shape: (75000, 11)
Test embeddings shape: (75000, 1536)
Test tabular shape: (75000, 9)

✅ Train final shape: (75000, 1547)
✅ Test final shape: (75000, 1545)


In [42]:
# ✅ Align test_tabular columns to train_tabular
missing_cols = set(train_tabular.columns) - set(test_tabular.columns)
extra_cols = set(test_tabular.columns) - set(train_tabular.columns)

print("Missing in test:", missing_cols)
print("Extra in test:", extra_cols)

# Add missing columns as 0
for col in missing_cols:
    test_tabular[col] = 0

# Drop any extra columns (if exist)
test_tabular = test_tabular[train_tabular.columns]

# ✅ Recombine final arrays
X_train_final = np.hstack([train_embeddings, train_tabular.values])
X_test_final = np.hstack([test_embeddings, test_tabular.values])

print("\nAligned Train shape:", X_train_final.shape)
print("Aligned Test shape:", X_test_final.shape)


Missing in test: {'price_per_unit', 'quantity_count', 'quantity_standard'}
Extra in test: {'text_all'}

Aligned Train shape: (75000, 1547)
Aligned Test shape: (75000, 1547)


In [44]:
# Only numeric tabular features + CNN embeddings
X_train_final = np.hstack([train_embeddings, train_tabular[["quantity_numeric","value"]].values])
X_test_final  = np.hstack([test_embeddings, test_tabular[["quantity_numeric","value"]].values])

print("Train final shape:", X_train_final.shape)  # should match model trained shape
print("Test final shape:", X_test_final.shape)    # must match train


Train final shape: (75000, 1538)
Test final shape: (75000, 1538)


In [45]:
print("\n🔮 Generating predictions... (this may take a few minutes)")

cat_preds = cat_model.predict(X_test_final)
lgb_preds = lgb_model.predict(X_test_final)
xgb_preds = xgb_model.predict(X_test_final)

# Weighted ensemble
weights = {"CatBoost": 0.2, "LightGBM": 0.4, "XGBoost": 0.4}
final_preds = (
    weights["CatBoost"] * cat_preds +
    weights["LightGBM"] * lgb_preds +
    weights["XGBoost"] * xgb_preds
)

# Reverse log transform (if trained on log)
final_preds = np.expm1(final_preds)

# Save submission
submission = pd.DataFrame({
    "sample_id": test_tabular.index,  # or use original test_df["sample_id"]
    "predicted_price": final_preds
})
submission.to_csv("final_predictions_with_image.csv", index=False)

print("✅ Final predictions saved to final_predictions_with_image.csv")



🔮 Generating predictions... (this may take a few minutes)


CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:72: Feature 1538 is present in model but not in pool.

In [46]:
import numpy as np
import pandas as pd

# --- Assume train_tabular and test_tabular are loaded ---
train_cols = train_tabular.columns
test_cols  = test_tabular.columns

# Add missing columns to test with zeros (or a default value)
for col in set(train_cols) - set(test_cols):
    test_tabular[col] = 0

# Drop extra columns from test
test_tabular = test_tabular[train_cols]

# Combine embeddings + numeric tabular
X_train_final = np.hstack([train_embeddings, train_tabular.values])
X_test_final  = np.hstack([test_embeddings, test_tabular.values])

print("Aligned Train shape:", X_train_final.shape)
print("Aligned Test shape:", X_test_final.shape)


Aligned Train shape: (75000, 1547)
Aligned Test shape: (75000, 1547)
